In [16]:
from linkers.context_aware import ContextAwareLinker 
from collections import defaultdict
from candidate import Candidate
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from candidate import Phrase, make_phrases
import re
from pandas import read_csv 

       
class DenseLinker(ContextAwareLinker):
    def foo(self):
        pass
    
    def _vectorize_texts(self, texts):
        # encode all these using sentence embeddings as well and/or average word2vec

        # load a gensim model

        # tokenize the words
        # average the words

        return 
 
class SparseLiker(ContextAwareLinker):
    def __init__(self, tfidf=True):
        ContextAwareLinker.__init__(self)
        self._tfidf = tfidf
          
    def train(self, dataset_fpaths):
        phrases = self._dataset2phrases(dataset_fpaths)
        phrase2candidates = self._train(phrases)
    
        candidates = set()
        for phrase in phrase2candidates:
            for candidate in phrase2candidates[phrase]:
                candidates.add(candidate)
        print(len(candidates))
    
        return candidates 
    
    def _train(self, phrases):
        candidates = self.get_candidates(phrases)
        return candidates
        # create a vectorizable dictionary from the candidates 
        
        #vectorizer = TfidfVectorizer() if tfidf else CountVectorizer()
        
        
    # to an intermetidate TrainableLinker class?
    def _dataset2phrases(self, dataset_fpaths):
        voc = set()
        for dataset_fpath in dataset_fpaths:
            df = read_csv(dataset_fpath, sep="\t", encoding="utf-8")
            for i, row in df.iterrows():
                for target in str(row.targets).split(","):
                    voc.add(target.strip())
            
        return make_phrases(list(voc))
        
    def link(self, context, phrases):
        #X_train_counts = count_vect.fit_transform(twenty_train.data)
    
    
# l = ContextAwareLinker()
# c = l.get_candidates(make_dummy_phrases(["San Francisco"]))
dataset_fpaths = ["datasets/dbpedia.tsv", "datasets/kore50.tsv", "datasets/n3-reuters-128.tsv"]

sl = SparseLiker()
c = sl.train(dataset_fpaths)

 15%|█▍        | 149/997 [03:49<21:47,  1.54s/it]

Traceback (most recent call last):
  File "/home/panchenko/kb2vec/converter.py", line 63, in wikidataid2wikipedia
    entity = self._client.get(wikidata_q_id, load=True)
  File "/home/panchenko/anaconda/lib/python3.6/site-packages/wikidata/client.py", line 139, in get
    entity.load()
  File "/home/panchenko/anaconda/lib/python3.6/site-packages/wikidata/entity.py", line 239, in load
    result = self.client.request(url)
  File "/home/panchenko/anaconda/lib/python3.6/site-packages/wikidata/client.py", line 193, in request
    response = self.opener.open(url)
  File "/home/panchenko/anaconda/lib/python3.6/urllib/request.py", line 532, in open
    response = meth(req, response)
  File "/home/panchenko/anaconda/lib/python3.6/urllib/request.py", line 642, in http_response
    'http', request, response, code, msg, hdrs)
  File "/home/panchenko/anaconda/lib/python3.6/urllib/request.py", line 570, in error
    return self._call_chain(*args)
  File "/home/panchenko/anaconda/lib/python3.6/urlli

 26%|██▌       | 259/997 [05:50<16:38,  1.35s/it]

 47%|████▋     | 468/997 [10:17<11:37,  1.32s/it]

 51%|█████▏    | 511/997 [11:21<10:48,  1.33s/it]

...

 86%|████████▌ | 855/997 [18:11<03:01,  1.28s/it]

 94%|█████████▍| 941/997 [20:36<01:13,  1.31s/it]

Traceback (most recent call last):
  File "/home/panchenko/kb2vec/converter.py", line 63, in wikidataid2wikipedia
    entity = self._client.get(wikidata_q_id, load=True)
  File "/home/panchenko/anaconda/lib/python3.6/site-packages/wikidata/client.py", line 139, in get
    entity.load()
  File "/home/panchenko/anaconda/lib/python3.6/site-packages/wikidata/entity.py", line 239, in load
    result = self.client.request(url)
  File "/home/panchenko/anaconda/lib/python3.6/site-packages/wikidata/client.py", line 193, in request
    response = self.opener.open(url)
  File "/home/panchenko/anaconda/lib/python3.6/urllib/request.py", line 532, in open
    response = meth(req, response)
  File "/home/panchenko/anaconda/lib/python3.6/urllib/request.py", line 642, in http_response
    'http', request, response, code, msg, hdrs)
  File "/home/panchenko/anaconda/lib/python3.6/urllib/request.py", line 570, in error
    return self._call_chain(*args)
  File "/home/panchenko/anaconda/lib/python3.6/urlli

100%|██████████| 997/997 [21:20<00:00,  1.28s/it]


103227


In [ ]:
import codecs
from candidate import make_dummy_phrases 


output_fpath = "data/sf-candidates.txt"
re_newlines = re.compile(r"[\n\r]+")

with codecs.open(output_fpath, "w", "utf-8") as c_f:
    for phrase in c:
        for candidate in c[phrase]:
            text = candidate.text
            c_f.write("{}\t{}\t{}\n".format(
                phrase.text,
                candidate.name,
                text.strip()))
            
print(output_fpath)

In [40]:
# from utils import ensure_dir 
from os.path import join 
from sklearn.externals import joblib
import json 
import os


def ensure_dir(f):
    """ Make the directory. """
    
    if not os.path.exists(f):
        os.makedirs(f)
        return True
    else:
        return False

        
candidates = list(c)
model_dir = "datasets/103227/"

# loading ...
# clf = joblib.load('filename.pkl') 
# with open('data.json', 'r') as fp:
#     data = json.load(fp)


# make self constants in constructor 
vectorizer_filename = "vectorizer"
candidate2index_filename = "candidate2index.pkl"
params_filename = "params.json"
vectors_filename = "vectors"
tfidf = True # instead self._params["tfidf"] = True

vectorizer_fpath = join(model_dir, vectorizer_filename)
candidate2index_fpath = join(model_dir, candidate2index_filename)
params_fpath = join(model_dir, params_filename) 
vectors_fpath = join(model_dir, vectors_filename)

# prepare the data
candidate2index = {}
corpus = []
for index, candidate in enumerate(candidates):
    corpus.append(candidate.text)
    candidate2index[candidate] = index
    
# get the vectors
vectorizer = TfidfVectorizer() if tfidf else CountVectorizer()
vectors = vectorizer.fit_transform(corpus)

# save the results

print(ensure_dir(model_dir)) # make it in the constructor 

joblib.dump(vectors, vectors_fpath)
print("vectors:", vectors_fpath)

joblib.dump(vectorizer, vectorizer_fpath) 
print("vectorizer:", vectorizer_fpath)

joblib.dump(candidate2index, candidate2index_fpath)
print("candidate2index:", candidate2index_fpath)

# not here but in constructor ...
# with open(params_fpath, "w") as fp:
#    json.dump(self._params, fp)
print("params:", params_fpath)


False
vectors: datasets/103227/vectors
vectorizer: datasets/103227/vectorizer
candidate2index: datasets/103227/candidate2index.pkl
params: datasets/103227/params.json


In [25]:
import codecs

output_fpath = "data/997-phrases.txt"

with codecs.open(output_fpath, "w", "utf-8") as out:
    for candidate in c:
        out.write("{}\t{}\n".format(candidate.name, candidate.text))

print(output_fpath)    

data/997-phrases.txt


In [ ]:
from linkers.baseline import BaselineLinker
from candidate import Phrase

context = "San Francisco said the visit would serve as a cornerstone for future interaction between players and coaches from the Nets and young Russians, with the aim of developing basketball in Russia, where the sport is a distant third in popularity behind soccer and hockey."
phrases = "San Francisco"

phrases =  [Phrase(phrase.strip(), 1, len(phrase.strip()), "http://" + phrase.strip())
                   for phrase in phrases.split(",")]
bl = BaselineLinker()

for phrase, candidate in bl.link(context, phrases):
    print(phrase.text, candidate)

In [ ]:
from linkers.baseline import BaselineLinker
from candidate import Phrase
from pandas import read_csv 

dataset_fpath = "datasets/dbpedia.tsv"

df = read_csv(dataset_fpath, sep="\t", encoding="utf-8")
bl = BaselineLinker()

for i, row in df.iterrows():
    phrases =  [Phrase(phrase.strip(), 1, len(phrase.strip()), "http://" + phrase.strip())
                       for phrase in row.targets.split(",")]
        
    print("\n\n{}\n".format(row.context))
    
    for phrase, candidate in bl.link(row.context, phrases):
        link = candidate.link if candidate else ""
        print(phrase.text, link)

In [1]:
import tqmd

ModuleNotFoundError: No module named 'tqmd'

In [ ]:
%load_ext autoreload
%autoreload 2